In [1]:
#imports
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from wordcloud import WordCloud, STOPWORDS
%matplotlib inline

plt.rcParams['figure.figsize'] = [20, 10]

In [2]:
#loading dataset
df = pd.read_csv('Data/ingredientsV1.csv', sep=",")

In [3]:
df.head(30)

,id,brand,name,ingredients,weight
0,AVphBRHOilAPnD_x0OrE,Simon Fischer,Simon Fischer Fruit Bttr Prune Lekvar,"Dried Prunes,Water,Corn Syrup,Sugar,Pectin.",10.6 pounds
1,AVpfNFy1LJeJML434ma2,McCormick,McCORMICK GRILL MATES MOLASSES BACON SEASONING...,"Salt,Sugar,Molasses (Refinery Syrup, Molasses,...",3.5 ounces
2,AVpgT49VLJeJML43MJEz,Jolly Time,Jolly Time Popcorn,"Salt, Yellow 5 Lake, Tricalcium Phosphate And ...",1.8 pounds
3,AVphYgnzLJeJML43aPp2,Ziyad,Ziyad Tahini Sesame Sauce,Mechanically hulled seasame seeds.Allergy Info...,1.6 pounds
4,AVpiS0bOLJeJML43kRsh,Fla-Vor-Ice,Fla-Vor-Ice Plus Giant Pops,FALSE,18 pounds
5,AVpfiMykilAPnD_xdedK,Hero,Hero Fruit Sprd Blk Currant-12 Oz -pack of 8,"Red Raspberries,Sugar,Glucose Syrup,Citric Aci...",3.5 lbs
6,AVpgPmxs1cnluZ0-ypMt,Simply Asia,Simply Asia Noodle Bowl Mandarin Orange -- 8.5 oz,"Noodles: wheat flour,water,wheat gluten,modifi...",3.4 pounds
7,AVphcTTBLJeJML43a9fO,EMERIL S,Italian Bread Crumbs,"Wheat Flour,Soybean Oil,Salt,Dehydrated Garlic...",NaN
8,AVpfwHjULJeJML43DEtY,Back to Nature,BACK TO NATURE CEREAL ORANGE CRUNCH GRAN,"Rolled Oats,Brown Rice Syrup,Evaporated Cane J...",4.7 pounds
9,AVphhxMQ1cnluZ0-Dif0,Nutrament,Nutrament Strawberry Complete Nutrition Drink ...,"Skim Milk,Sugar,Corn Syrup,Canola Oil,High Ole...",NaN


In [4]:
df.shape

(10000, 5)

In [5]:
df["brand"].value_counts()

Goya Food              77
CVS Gold Emblem        75
Roland                 75
Bob's Red Mill         71
Smart Sense            70
                       ..
PapaNicholas Coffee     1
Loc Maria               1
Annie Chun's            1
Yan Yan                 1
Just Like Sugar         1
Name: brand, Length: 3143, dtype: int64

EDA

Transform ingredients in list

In [6]:
df["ingredients"]

0             Dried Prunes,Water,Corn Syrup,Sugar,Pectin.
1       Salt,Sugar,Molasses (Refinery Syrup, Molasses,...
2       Salt, Yellow 5 Lake, Tricalcium Phosphate And ...
3       Mechanically hulled seasame seeds.Allergy Info...
4                                                   FALSE
                              ...                        
9995                  water,natural hickory smoke flavor.
9996    Organic Dry Roasted Almonds,Organic Tamari Soy...
9997    Cashews, Vegetable Oil (May Contain One Or Mor...
9998    Whole Grain Brown Rice Flour,Degerminated Mill...
9999    Non-GMO Whole Grain Wheat,Non-GMO Whole Grain ...
Name: ingredients, Length: 10000, dtype: object

Transform data in lower case data

In [7]:
df['ingredients_lower'] = df["ingredients"].str.lower()

In [8]:
df['ingredients_lower']

0             dried prunes,water,corn syrup,sugar,pectin.
1       salt,sugar,molasses (refinery syrup, molasses,...
2       salt, yellow 5 lake, tricalcium phosphate and ...
3       mechanically hulled seasame seeds.allergy info...
4                                                   false
                              ...                        
9995                  water,natural hickory smoke flavor.
9996    organic dry roasted almonds,organic tamari soy...
9997    cashews, vegetable oil (may contain one or mor...
9998    whole grain brown rice flour,degerminated mill...
9999    non-gmo whole grain wheat,non-gmo whole grain ...
Name: ingredients_lower, Length: 10000, dtype: object

Remove innecesary characters

In [9]:
df['ingredients_no_special_char'] = df['ingredients_lower'].apply(lambda x: re.sub('[^,a-zA-Z ]+', '', str(x)))
df['ingredients_no_special_char'] 

0              dried prunes,water,corn syrup,sugar,pectin
1       salt,sugar,molasses refinery syrup, molasses, ...
2       salt, yellow  lake, tricalcium phosphate and a...
3       mechanically hulled seasame seedsallergy infor...
4                                                   false
                              ...                        
9995                   water,natural hickory smoke flavor
9996    organic dry roasted almonds,organic tamari soy...
9997    cashews, vegetable oil may contain one or more...
9998    whole grain brown rice flour,degerminated mill...
9999    nongmo whole grain wheat,nongmo whole grain oa...
Name: ingredients_no_special_char, Length: 10000, dtype: object

Transform to a list

In [10]:
df['ingredients_list'] = df['ingredients_no_special_char'].str.split(',')

In [11]:
df['ingredients_list']

0        [dried prunes, water, corn syrup, sugar, pectin]
1       [salt, sugar, molasses refinery syrup,  molass...
2       [salt,  yellow  lake,  tricalcium phosphate an...
3       [mechanically hulled seasame seedsallergy info...
4                                                 [false]
                              ...                        
9995                [water, natural hickory smoke flavor]
9996    [organic dry roasted almonds, organic tamari s...
9997    [cashews,  vegetable oil may contain one or mo...
9998    [whole grain brown rice flour, degerminated mi...
9999    [nongmo whole grain wheat, nongmo whole grain ...
Name: ingredients_list, Length: 10000, dtype: object

In [12]:
def removeSpace(x):
    return [y.strip() for y in x]

In [13]:
df['ingredients_list'].apply(removeSpace)

0        [dried prunes, water, corn syrup, sugar, pectin]
1       [salt, sugar, molasses refinery syrup, molasse...
2       [salt, yellow  lake, tricalcium phosphate and ...
3       [mechanically hulled seasame seedsallergy info...
4                                                 [false]
                              ...                        
9995                [water, natural hickory smoke flavor]
9996    [organic dry roasted almonds, organic tamari s...
9997    [cashews, vegetable oil may contain one or mor...
9998    [whole grain brown rice flour, degerminated mi...
9999    [nongmo whole grain wheat, nongmo whole grain ...
Name: ingredients_list, Length: 10000, dtype: object

In [14]:
def removeStopWords(lst_ingredients):
    ingredients2 = []
    receipe = []
    for index, ingredients in enumerate(lst_ingredients):
        ing_word = ingredients.split(" ")
        ing_word = [i for i in ing_word if i not in STOPWORDS]
        ingredients2.append(" ".join(ing_word))
    ingredients = ingredients2[:]
    return ingredients

In [15]:
df['ingredients_list'].apply(removeStopWords)

0        [dried prunes, water, corn syrup, sugar, pectin]
1       [salt, sugar, molasses refinery syrup,  molass...
2       [salt,  yellow  lake,  tricalcium phosphate ar...
3       [mechanically hulled seasame seedsallergy info...
4                                                 [false]
                              ...                        
9995                [water, natural hickory smoke flavor]
9996    [organic dry roasted almonds, organic tamari s...
9997    [cashews,  vegetable oil may contain one follo...
9998    [whole grain brown rice flour, degerminated mi...
9999    [nongmo whole grain wheat, nongmo whole grain ...
Name: ingredients_list, Length: 10000, dtype: object

In [16]:
ingredients_dict = {}
for recipe in df['ingredients_list']:
    #print(recipe)
    for ingredient in recipe:
        #print(ingredient.strip())
        ingredients_dict[ingredient.strip()] = ingredients_dict.get(ingredient.strip(),0)+ 1
        
ingredients_dict

{'dried prunes': 1,
 'water': 2472,
 'corn syrup': 1231,
 'sugar': 4164,
 'pectin': 151,
 'salt': 5250,
 'molasses refinery syrup': 1,
 'molasses': 326,
 'caramel color': 657,
 'spices including black pepper': 7,
 'garlic onion': 1,
 'tapioca maltodextrin': 14,
 'bacon fat and cooked bacon cured with water': 1,
 'sodium erythorbate': 117,
 'sodium nitrate': 19,
 'silicon dioxide to make free flowing': 2,
 'autolyzed yeast': 45,
 'sunflower oil': 319,
 'corn maltodextrin': 25,
 'vinegar': 359,
 'extractives of paprika': 49,
 'and natural flavor including smoke': 1,
 'yellow  lake': 319,
 'tricalcium phosphate and artificial butter flavor': 1,
 'mechanically hulled seasame seedsallergy information packed in a facility that processes wheat': 2,
 'flour': 39,
 'peanuts and tree nuts': 7,
 'false': 1,
 'red raspberries': 8,
 'glucose syrup': 124,
 'citric acid': 2034,
 'pectin contains wheat': 1,
 'noodles wheat flour': 5,
 'wheat gluten': 253,
 'modified tapioca starch': 36,
 'sodium algin

In [17]:
ing_df = pd.DataFrame(data = ingredients_dict.values(),index = ingredients_dict.keys(),columns = ["Counts"])
ing_df.sort_values(["Counts"],ascending = False, inplace = True)
ing_df

,Counts
salt,5250
sugar,4164
water,2472
citric acid,2034
folic acid,1478
...,...
spice herb,1
quinoa seeds contains milk,1
enriched ground white corn corn flour,1
enriched masa flour white corn flour,1


In [18]:
ingredients_map = {k:v for k,v in zip(ing_df.index,range(len(ing_df)))}
ingredients_map

{'salt': 0,
 'sugar': 1,
 'water': 2,
 'citric acid': 3,
 'folic acid': 4,
 'niacin': 5,
 'soy lecithin': 6,
 'corn syrup': 7,
 'natural flavor': 8,
 '': 9,
 'dextrose': 10,
 'yellow': 11,
 'reduced iron': 12,
 'natural flavors': 13,
 'riboflavin': 14,
 'high fructose corn syrup': 15,
 'sea salt': 16,
 'maltodextrin': 17,
 'cocoa butter': 18,
 'spices': 19,
 'red': 20,
 'soybean oil': 21,
 'caramel color': 22,
 'blue': 23,
 'enzymes': 24,
 'xanthan gum': 25,
 'natural and artificial flavors': 26,
 'thiamine mononitrate': 27,
 'wheat flour': 28,
 'corn starch': 29,
 'artificial flavor': 30,
 'palm oil': 31,
 'yeast': 32,
 'sodium citrate': 33,
 'riboflavin vitamin b': 34,
 'garlic': 35,
 'baking soda': 36,
 'milk': 37,
 'thiamin mononitrate': 38,
 'chocolate': 39,
 'natural and artificial flavor': 40,
 'modified corn starch': 41,
 'iron': 42,
 'whey': 43,
 'garlic powder': 44,
 'onion powder': 45,
 'honey': 46,
 'monocalcium phosphate': 47,
 'lactic acid': 48,
 'canola oil': 49,
 'vineg

df

In [19]:
df.head()

,id,brand,name,ingredients,weight,ingredients_lower,ingredients_no_special_char,ingredients_list
0,AVphBRHOilAPnD_x0OrE,Simon Fischer,Simon Fischer Fruit Bttr Prune Lekvar,"Dried Prunes,Water,Corn Syrup,Sugar,Pectin.",10.6 pounds,"dried prunes,water,corn syrup,sugar,pectin.","dried prunes,water,corn syrup,sugar,pectin","[dried prunes, water, corn syrup, sugar, pectin]"
1,AVpfNFy1LJeJML434ma2,McCormick,McCORMICK GRILL MATES MOLASSES BACON SEASONING...,"Salt,Sugar,Molasses (Refinery Syrup, Molasses,...",3.5 ounces,"salt,sugar,molasses (refinery syrup, molasses,...","salt,sugar,molasses refinery syrup, molasses, ...","[salt, sugar, molasses refinery syrup, molass..."
2,AVpgT49VLJeJML43MJEz,Jolly Time,Jolly Time Popcorn,"Salt, Yellow 5 Lake, Tricalcium Phosphate And ...",1.8 pounds,"salt, yellow 5 lake, tricalcium phosphate and ...","salt, yellow lake, tricalcium phosphate and a...","[salt, yellow lake, tricalcium phosphate an..."
3,AVphYgnzLJeJML43aPp2,Ziyad,Ziyad Tahini Sesame Sauce,Mechanically hulled seasame seeds.Allergy Info...,1.6 pounds,mechanically hulled seasame seeds.allergy info...,mechanically hulled seasame seedsallergy infor...,[mechanically hulled seasame seedsallergy info...
4,AVpiS0bOLJeJML43kRsh,Fla-Vor-Ice,Fla-Vor-Ice Plus Giant Pops,FALSE,18 pounds,false,false,[false]


In [26]:
df.ingredients_list.head(20)

0      [dried prunes, water, corn syrup, sugar, pectin]
1     [salt, sugar, molasses refinery syrup,  molass...
2     [salt,  yellow  lake,  tricalcium phosphate an...
3     [mechanically hulled seasame seedsallergy info...
4                                               [false]
5     [red raspberries, sugar, glucose syrup, citric...
6     [noodles wheat flour, water, wheat gluten, mod...
7     [wheat flour, soybean oil, salt, dehydrated ga...
8     [rolled oats, brown rice syrup, evaporated can...
9     [skim milk, sugar, corn syrup, canola oil, hig...
10    [filtered water, whole organic soybeans, organ...
11    [filtered water, whole organic soybeans, organ...
12    [almondmilk filtered water,  almonds, evaporat...
13    [enriched flour wheat flour,  malted barley fl...
14    [anise seed used mainly to add flavor to cordi...
15    [garlic powder  teaspoon garlic powder   mediu...
16    [sugar, corn syrup, gelatin, citric acid, lact...
17    [corn, partially hydrogenated soybean ando

Convert ingredients to numbers

In [20]:
ing_df.shape[0]

21481

In [21]:
def convert_recipe(recipe):
    output = np.zeros(ing_df.shape[0])
    for ingredient in recipe:
        output[ingredients_map[ingredient.strip()]] = 1
    return output

    
df["Vector"] = df["ingredients_list"].apply(convert_recipe) # Convert each recipe to a ONE Sparse Vector Form

In [22]:
df["Vector"]

0       [0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...
1       [1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2       [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4       [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
                              ...                        
9995    [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
9996    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
9997    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
9998    [1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...
9999    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: Vector, Length: 10000, dtype: object

In [23]:
#Store all the vectors as a Matrix of M x 7137
mat = list(df["Vector"])
mat = np.array(mat)
mat.shape

(10000, 21481)

PCA

In [24]:
from sklearn.decomposition import PCA
pca = PCA(32)
mat_32 = pca.fit_transform(mat)

In [25]:
mat_32

array([[ 0.08213076,  0.52186073,  0.09357403, ..., -0.08808011,
        -0.03794582, -0.08799017],
       [ 0.36720915,  0.4649142 , -0.23944715, ...,  0.24007443,
        -0.12841871, -0.21120497],
       [-0.19316458, -0.0194377 , -0.26589629, ..., -0.03704916,
        -0.08082701, -0.0047333 ],
       ...,
       [-0.67919164, -0.2475711 ,  0.02943872, ..., -0.22566866,
        -0.11810432, -0.08313179],
       [ 0.32150726,  0.46581794, -0.6362054 , ...,  0.04893462,
         0.21903984,  0.1777551 ],
       [-0.62857904, -0.20596967,  0.06965235, ...,  0.29167647,
         0.07088524,  0.09522608]])